In [1]:
# -*- coding: utf-8 -*-
import sys
import os
import numpy as np
import pandas as pd
import cobra

print('Python version:', sys.version)
print('numpy version:', np.__version__)
print('pandas version:', pd.__version__)
print('cobrapy version:', cobra.__version__)

Python version: 3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)]
numpy version: 1.16.4
pandas version: 1.0.3
cobrapy version: 0.15.4


In [2]:
def AddRxn(model, newRxnFile):
    """Function of adding new reactions to the model."""
    n1 = len(model.reactions)
    AllAddRxn = pd.read_csv(newRxnFile, sep=',', index_col='RxnID', skipinitialspace=True)
    n2 = len(AllAddRxn)
    for i in range(n2):
        ID = AllAddRxn.index.values[i]
        addRxn = cobra.Reaction(ID)
        model.add_reactions([addRxn])
        addRxnInf = model.reactions[n1 + i]
        addRxnInf.name = AllAddRxn.loc[ID, 'RxnName']
        addRxnInf.reaction = AllAddRxn.loc[ID, 'RxnFormula']
        addRxnInf.subsystem = AllAddRxn.loc[ID, 'Subsystem']
        addRxnInf.lower_bound = AllAddRxn.loc[ID, 'LowerBound']
        addRxnInf.upper_bound = AllAddRxn.loc[ID, 'UpperBound']
    return model


In [3]:
def flux2file(model, product, psw, output_dir='tmp'):
    """Function of exporting flux data."""
    n = len(model.reactions)
    modelMatrix = np.empty([n, 9], dtype = object)
    for i in range(len(model.reactions)):
        x = model.reactions[i]
        modelMatrix[i, 0] = i + 1
        modelMatrix[i, 1] = x.id
        modelMatrix[i, 2] = x.name
        modelMatrix[i, 3] = x.reaction
        modelMatrix[i, 4] = x.subsystem
        modelMatrix[i, 5] = x.lower_bound
        modelMatrix[i, 6] = x.upper_bound
        modelMatrix[i, 7] = x.flux
        modelMatrix[i, 8] = abs(x.flux)
        
    df = pd.DataFrame(data = modelMatrix, 
                        columns = ['N', 'RxnID', 'RxnName', 'Reaction', 'SubSystem', 
                        'LowerBound', 'UpperBound', 'Flux-core', 'abs(Flux)'])
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    filepath = os.path.join(output_dir, '{}_{}.xlsx'.format(product, psw))
    df.to_excel(filepath, index=False)
    

In [4]:
model = cobra.Model()

AddRxn(model,'CBBrxns.csv')

# the model has a constraint of producing 1 of 3pg (sink_3pg)
# set the objective to minimize ATP cost
model.objective = {model.reactions.DM_atp: 1}
model.objective_direction = 'min'

# set the carboxylation and oxygenation reaction ratio 
# of RuBisCO to be 3:1
rubisco_flux = model.problem.Constraint(
    model.reactions.RBPC.flux_expression - 3 * model.reactions.RBPO.flux_expression,
    lb = 0, 
    ub = 0
)
model.add_cons_vars(rubisco_flux)

unknown metabolite 'co2' created
unknown metabolite 'h2o' created
unknown metabolite 'rb15bp' created
unknown metabolite '3pg' created
unknown metabolite 'h' created
unknown metabolite 'o2' created
unknown metabolite '2pglyc' created
unknown metabolite 'atp' created
unknown metabolite '13dpg' created
unknown metabolite 'adp' created
unknown metabolite 'nadph' created
unknown metabolite 'g3p' created
unknown metabolite 'nadp' created
unknown metabolite 'pi' created
unknown metabolite 'dhap' created
unknown metabolite 'fdp' created
unknown metabolite 'f6p' created
unknown metabolite 'e4p' created
unknown metabolite 'xu5p' created
unknown metabolite 's17bp' created
unknown metabolite 's7p' created
unknown metabolite 'r5p' created
unknown metabolite 'ru5p' created
unknown metabolite 'amp' created
unknown metabolite 'nad' created
unknown metabolite 'nadh' created
unknown metabolite 'fdxo' created
unknown metabolite 'fdxrd' created
unknown metabolite 'ppi' created
unknown metabolite 'hco3' c

In [5]:
photores = {
    'NPR': 'a_NPRrxns.csv',    # natural photorespiration
    'GLC': 'b_GLCrxns.csv',    # glycerate bypass
    'OX': 'c_OXrxns.csv',      # glycolate oxidation pathway
    'A5P': 'd_A5Prxns.csv',    # arabinose-5-phosphate shunt
    '3OHP': 'e_3OHPrxns.csv',  # 3-hydroxypropionate bypass
    'TACO': 'f_TACOrxns.csv',  # tartronyl-CoA pathway
}

cost_df = pd.DataFrame()

In [6]:
for psw, rxns in photores.items():
    with model as m:
        AddRxn(m, rxns)
        m.optimize()
        flux2file(m,'3pg',psw,'output')
        for cost in ['DM_atp', 'DM_e', 'Fdr', 'EX_co2', 'RBPC', 'RBPO']:
            cost_df.loc[cost, psw] = abs(m.reactions.get_by_id(cost).flux)
    

unknown metabolite 'glyclt' created
unknown metabolite 'glx' created
unknown metabolite 'h2o2' created
unknown metabolite 'ser' created
unknown metabolite 'gly' created
unknown metabolite 'hpyr' created
unknown metabolite 'glu' created
unknown metabolite 'akg' created
unknown metabolite 'gln' created
unknown metabolite 'mlthf' created
unknown metabolite 'thf' created
unknown metabolite 'glyc' created
unknown metabolite 'glyclt' created
unknown metabolite 'glx' created
unknown metabolite '2h3oppan' created
unknown metabolite 'glyc' created
unknown metabolite 'glyclt' created
unknown metabolite 'glx' created
unknown metabolite 'h2o2' created
unknown metabolite 'accoa' created
unknown metabolite 'mal' created
unknown metabolite 'coa' created
unknown metabolite 'pyr' created
unknown metabolite 'glyclt' created
unknown metabolite 'coa' created
unknown metabolite 'glyccoa' created
unknown metabolite 'gcald' created
unknown metabolite 'ara5p' created
unknown metabolite 'glyclt' created
unknow

In [7]:
# Assuming we cannot improve the GCC hydrolysis
# The GCC M5 hydrolyse 3.9 ATP per carboxylation (Fig. 2c)
with model as m:
    AddRxn(m, 'f_TACOrxns.csv')
    m.reactions.GCC.add_metabolites({'atp': -2.9, 'adp': 2.9, 'pi': 2.9})
    m.optimize()
    flux2file(m,'3pg', 'TACO_2','output')
    for cost in ['DM_atp', 'DM_e', 'Fdr', 'EX_co2', 'RBPC', 'RBPO']:
        cost_df.loc[cost, 'TACO_2'] = abs(m.reactions.get_by_id(cost).flux)

unknown metabolite 'glyclt' created
unknown metabolite 'coa' created
unknown metabolite 'glyccoa' created
unknown metabolite 'tarcoa' created
unknown metabolite '2h3oppan' created
unknown metabolite 'glyc' created


In [8]:
cost_df.to_excel('TaCo costs comparison.xlsx')
cost_df

,NPR,GLC,OX,A5P,3OHP,TACO,TACO_2
DM_atp,14.0,13.4,32.0,12.0,11.75,11.00,13.175
DM_e,8.0,7.4,14.0,7.0,6.50,6.50,6.500
Fdr,1.2,0.0,0.0,0.0,0.00,0.00,0.000
EX_co2,3.0,3.0,3.0,3.0,3.00,3.00,3.000
RBPC,3.6,3.6,9.0,3.0,2.25,2.25,2.250
RBPO,1.2,1.2,3.0,1.0,0.75,0.75,0.750
